In [ ]:
#!git clone https://github.com/zhouhaoyi/Informer2020.git
#!git clone https://github.com/zhouhaoyi/ETDataset.git
# modify the code based on my dataset

In [ ]:
# !{sys.executable} -m pip install matplotlib
# !{sys.executable} -m pip install pandas
# !{sys.executable} -m pip install scikit_learn
# !{sys.executable} -m pip install torch
# !{sys.executable} -m pip install seaborn

In [1]:
import sys
import os
import pandas as pd
import numpy as np
from argparse import Namespace

In [2]:
if not 'Informer2020' in sys.path:
    sys.path += ['Informer2020']

from data.data_loader import Dataset_Custom
from utils.metrics import metric

In [3]:
import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())

2.8.0+cu128
12.8
True


In [4]:
num_gpus = torch.cuda.device_count()
print("Number of GPUs available:", num_gpus)

for i in range(num_gpus):
    gpu_name = torch.cuda.get_device_name(i)
    print(f"GPU {i}: {gpu_name}")

Number of GPUs available: 1
GPU 0: NVIDIA A30


## Prepare the data

In [5]:
my_data = pd.read_csv('../GD030A_S.csv')

In [6]:
# Define the recover_timestamp function
def recover_timestamp(data):
    # Combine 'date' and 'time' to form a datetime column
    data['datetime'] = pd.to_datetime(data['date'] + ' ' + data['time'].astype(str) + ':00', format='%Y-%m-%d %H:%M')

    # Set 'datetime' as index
    data = data.set_index('datetime')

    # Create a complete range of timestamps with hourly frequency
    full_time_range = pd.date_range(start=data.index.min(), end=data.index.max(), freq='H')

    # Reindex the data to include all timestamps, filling missing rows with NaN
    data_full = data.reindex(full_time_range)

    return data_full

Custom data (xxx.csv) has to include at least 2 features: date(format: YYYY-MM-DD hh:mm:ss) and target feature.

In [ ]:
traffic_full = recover_timestamp(my_data)
traffic_full.drop(['date', 'time'], axis=1, inplace=True)
traffic_full = traffic_full.reset_index(names='date')
traffic_full['date'] = traffic_full['date'].astype(str)
traffic_full

In [ ]:
#traffic_full.to_csv('traffic_full.csv', index=False)

## Run training directly using bash

In [5]:
# ensure the shell commands below using the current Python environment
os.environ["PATH"] = f"{sys.prefix}/bin:" + os.environ["PATH"]

In [6]:
cd Informer2020

/home/yl2672496l/Yue/code/transformer/Informer2020


In [ ]:
#test
!python -u main_informer.py --model informer --data traffic_full --root_path ../ --features S --seq_len 24 --label_len 24 --pred_len 6 --e_layers 2 --d_layers 1 --attn prob --des 'Exp' --itr 1 #--inverse

In [9]:
seq_lens = [24 * i for i in range(1, 22)]
print(seq_lens)

[24, 48, 72, 96, 120, 144, 168, 192, 216, 240, 264, 288, 312, 336, 360, 384, 408, 432, 456, 480, 504]


In [ ]:
for seq_len in seq_lens:
    print(f"Running seq_len={seq_len}")
    !python -u main_informer.py --model informer --data traffic_full --root_path ../ --features S --seq_len {seq_len} --label_len 24 --pred_len 6 --e_layers 2 --d_layers 1 --attn prob --des 'Exp' --itr 10

Running seq_len=24
Args in experiment:
Namespace(model='informer', data='traffic_full', root_path='../', data_path='traffic_full.csv', features='S', target='flow', freq='h', checkpoints='./checkpoints/', seq_len=24, label_len=24, pred_len=6, enc_in=1, dec_in=1, c_out=1, d_model=512, n_heads=8, e_layers=2, d_layers=1, s_layers=[3, 2, 1], d_ff=2048, factor=5, padding=0, distil=True, dropout=0.05, attn='prob', embed='timeF', activation='gelu', output_attention=False, do_predict=False, mix=True, cols=None, num_workers=0, itr=10, train_epochs=6, batch_size=32, patience=3, learning_rate=0.0001, des='Exp', loss='mse', lradj='type1', use_amp=False, inverse=False, use_gpu=True, gpu=0, use_multi_gpu=False, devices='0,1,2,3', detail_freq='h')
Use GPU: cuda:0
>>>>>>>start training : informer_traffic_full_ftS_sl24_ll24_pl6_dm512_nh8_el2_dl1_df2048_atprob_fc5_ebtimeF_dtTrue_mxTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 24515
[DEBUG] Original dataset length: 24515
[DEBUG] Cleaned dataset length (NaN r

In [10]:
data = np.load("./results/running_time_24.npy")
data

array([17.32990809,  1.68137169])